<a href="https://colab.research.google.com/github/Tiagoalvesds/curry_company/blob/main/Analise_de_Risco_Carteira_Acoes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Bibliotecas**

In [ ]:
import pandas as pd
import numpy as np
import seaborn
import seaborn as sns
import yfinance
import yfinance as yf
from datetime import datetime

# **Construção da Carteira**

Importando dados da base de dado do  Yahoo finance

In [ ]:
ativo_1 = input('Insira o ativo com letra maiúscula seguido de .SA (Ex: PETR3.SA)......:')
ativo_2 = input('Insira o ativo com letra maiúscula seguido de .SA (Ex: VALE3.SA)......:')
ativo_3 = input('Insira o ativo com letra maiúscula seguido de .SA (Ex: BBDC4.SA)......:')
ativo_4 = input('Insira o ativo com letra maiúscula seguido de .SA (Ex: WEGE3.SA)......:')
ativo_5 = input('Insira o ativo com letra maiúscula seguido de .SA (Ex: ELET3.SA)......:')

ativos = [ativo_1, ativo_2, ativo_3, ativo_4, ativo_5]

pesos = np.array([0.2,0.2,0.2,0.2,0.2])

print(ativos)

Parâmetros

In [ ]:
data_inicio = input('Insira uma data inicial. (Ex. 2019-01-01) ......:')
data_fim = input('Insira uma data fim. (Ex. 2023-07-13)......:')

carteira = yf.download (ativos, start=data_inicio, end=data_fim)['Adj Close']
carteira

# **Retornos**

In [ ]:
retornos = carteira.pct_change()
retornos

In [ ]:
retornos.dropna(inplace=True)
retornos.head()

# Retorno do Portfólio
### Somatório do Peso x Retorno


In [ ]:
retorno_carteira = (retornos*pesos).sum(axis=1)
retorno_portfolio = pd.DataFrame()
retorno_portfolio ['Retornos'] = retorno_carteira
retorno_portfolio

# Estatística descritiva.

Um quartil é qualquer um dos três valores que divide o conjunto ordenado de dados em quatro partes iguais, e assim cada parte representa 1/4 da amostra.

Assim, no caso de uma amostra ordenada:

* primeiro quartil (designado por Q1/4) = quartil inferior = é o valor aos 25% da amostra ordenada = 25º percentil.

* segundo quartil (designado por Q2/4) = mediana = é o valor até ao qual se encontra 50% da amostra ordenada = 50º percentil, ou  5º decil.

* terceiro quartil (designado por Q3/4) = quartil superior = valor a partir do qual se encontram 25% dos valores mais elevados = valor aos 75% da amostra ordenada = 75º percentil à diferença entre os quartis superior e inferior chama-se amplitude inter-quartil.



In [ ]:
#Neste código iremos explorar os retornos da carteira através do Quartil.
retorno_portfolio.describe()


In [ ]:
retorno_portfolio.plot.hist(bins=120)

# Teoria moderna do portfólio - Markowtiz
A teoria do portfólio estabelece que decisões relacionadas à seleção de investimentos devam ser tomadas com base na relação risco-retorno. Para auxiliar neste processo, modelos de otimização de portfólio têm sido desenvolvidos. De modo a serem efetivos, tais modelos devem ser capazes de quantificar os níveis de risco e retorno dos investimentos.

In [ ]:
corr = retornos.corr()
sns.heatmap(corr, annot=True)


# Matriz de Covariância

In [ ]:
cov = retornos.cov()
cov

# Volatilidade da Carteira

In [ ]:
vol_carteira = np.sqrt(np.dot(pesos.T, np.dot(cov,pesos)))
vol_carteira

In [ ]:
vol_ano = vol_carteira*np.sqrt(252)
vol_ano

# Beta da Carteira
(mede a sensibilidade do retorno da carteira em relação a volatilidade do mercado)

In [ ]:
ibov = yf.download ('^BVSP',start=data_inicio, end=data_fim) ['Adj Close']
ibov.describe()

Histograma do IBOVESPA

In [ ]:
ibov.plot.hist(bins=60)

In [ ]:
ibov_retornos = ibov.pct_change()
ibov_retornos.dropna(axis = 0, inplace = True)
ibov_retornos

# Beta da Carteira

In [ ]:
beta_carteira = pd.merge(retorno_portfolio, ibov_retornos, how="inner", left_index=True, right_index= True)
beta_carteira.head()

# Calculando a regressão com Statsmodels

In [ ]:
import statsmodels
import statsmodels.api as sm

In [ ]:
Y = beta_carteira['Retornos']
X = beta_carteira['Adj Close']

X = sm.add_constant(X)
modelo = sm.OLS(Y,X)
resultado = modelo.fit()

# **Calculando o Alfa da carteira**

In [ ]:
resultado.params[0]

# **Calculando o Beta da carteira**

In [ ]:
resultado.params[1]

# **Value at Risk - VaR Histórico**

In [ ]:
retorno_carteira.plot.hist(bins=60)

# VaR Histórico
### O que exceder o VaR é a perda estimada no pior cenário

In [ ]:
var_95 = np.nanpercentile(retorno_portfolio,5)
var_98 = np.nanpercentile(retorno_portfolio,2)
var_99 = np.nanpercentile(retorno_portfolio,1)

print(var_95, var_98, var_99)
